In [1]:
import time

import kagglehub

# Download latest version
path = kagglehub.dataset_download("melzohbi/metaphor-detection-vua-wsd-augmented")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\allex\.cache\kagglehub\datasets\melzohbi\metaphor-detection-vua-wsd-augmented\versions\1


In [2]:
import os
import pandas as pd

train_df = pd.read_csv(os.path.join(path, "data/VUA20/train.tsv"), sep='\t')
test_df = pd.read_csv(os.path.join(path, "data/VUA20/test.tsv"), sep='\t')

# get only 617 elements because testing would take too long with the whole dataset

In [3]:
tmp_train_df = train_df[:100]

In [4]:
tmp_train_df

,index,label,sentence,POS,FGPOS,w_index,target,word_sense,definition
0,a1e-fragment01 1,0,Latest corporate unbundler reveals laid-back a...,ADJ,JJS,0,Latest,up to the immediate present; most recent or mo...,Near the end of a period of time.
1,a1e-fragment01 1,0,Latest corporate unbundler reveals laid-back a...,ADJ,JJ,1,corporate,of or belonging to a corporation,Of or relating to a corporation.
2,a1e-fragment01 1,0,Latest corporate unbundler reveals laid-back a...,PROPN,NNP,2,unbundler,unbundler,unbundler
3,a1e-fragment01 1,1,Latest corporate unbundler reveals laid-back a...,VERB,VBZ,3,reveals,make visible,To uncover; to show and display that which was...
4,a1e-fragment01 1,0,Latest corporate unbundler reveals laid-back a...,VERB,VBN,4,laid-back,laid-back,laid-back
...,...,...,...,...,...,...,...,...,...
95,a1e-fragment01 5,0,The 63-year-old head of Pembridge Investments ...,ADP,IN,31,at,at,at
96,a1e-fragment01 5,1,The 63-year-old head of Pembridge Investments ...,NOUN,NN,32,headquarters,(usually plural) the office that serves as the...,The military installation from which troops ar...
97,a1e-fragment01 5,0,The 63-year-old head of Pembridge Investments ...,DET,DT,34,the,the,the
98,a1e-fragment01 5,0,The 63-year-old head of Pembridge Investments ...,ADJ,JJR,35,more,(comparative of `much' used with mass nouns) a...,more


# import the model using Ollama - I am using deepseek-r1 with 14B (using all VRAM resources)

# Create prompt for each row from dataset

In [5]:
def create_prompt(row):
    return (
        f"Sentence: {row['sentence']}\n"
        f"Target Word: {row['target']} (POS: {row['POS']}, Word Index: {row['w_index']}, Word Sense: {row['word_sense']}, definition: {row['definition']})\n"
        "Is the target word used metaphorically? Provide a yes or no answer without explanation."
    )

tmp_train_df.loc[:, 'prompt'] = tmp_train_df.apply(create_prompt, axis=1)

C:\Users\allex\AppData\Local\Temp\ipykernel_2664\3510685946.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_train_df.loc[:, 'prompt'] = tmp_train_df.apply(create_prompt, axis=1)


# Get predictions

In [6]:
tmp_train_df['prediction'] = None

C:\Users\allex\AppData\Local\Temp\ipykernel_2664\668391106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_train_df['prediction'] = None


In [7]:
tmp_train_df

,index,label,sentence,POS,FGPOS,w_index,target,word_sense,definition,prompt,prediction
0,a1e-fragment01 1,0,Latest corporate unbundler reveals laid-back a...,ADJ,JJS,0,Latest,up to the immediate present; most recent or mo...,Near the end of a period of time.,Sentence: Latest corporate unbundler reveals l...,None
1,a1e-fragment01 1,0,Latest corporate unbundler reveals laid-back a...,ADJ,JJ,1,corporate,of or belonging to a corporation,Of or relating to a corporation.,Sentence: Latest corporate unbundler reveals l...,None
2,a1e-fragment01 1,0,Latest corporate unbundler reveals laid-back a...,PROPN,NNP,2,unbundler,unbundler,unbundler,Sentence: Latest corporate unbundler reveals l...,None
3,a1e-fragment01 1,1,Latest corporate unbundler reveals laid-back a...,VERB,VBZ,3,reveals,make visible,To uncover; to show and display that which was...,Sentence: Latest corporate unbundler reveals l...,None
4,a1e-fragment01 1,0,Latest corporate unbundler reveals laid-back a...,VERB,VBN,4,laid-back,laid-back,laid-back,Sentence: Latest corporate unbundler reveals l...,None
...,...,...,...,...,...,...,...,...,...,...,...
95,a1e-fragment01 5,0,The 63-year-old head of Pembridge Investments ...,ADP,IN,31,at,at,at,Sentence: The 63-year-old head of Pembridge In...,None
96,a1e-fragment01 5,1,The 63-year-old head of Pembridge Investments ...,NOUN,NN,32,headquarters,(usually plural) the office that serves as the...,The military installation from which troops ar...,Sentence: The 63-year-old head of Pembridge In...,None
97,a1e-fragment01 5,0,The 63-year-old head of Pembridge Investments ...,DET,DT,34,the,the,the,Sentence: The 63-year-old head of Pembridge In...,None
98,a1e-fragment01 5,0,The 63-year-old head of Pembridge Investments ...,ADJ,JJR,35,more,(comparative of `much' used with mass nouns) a...,more,Sentence: The 63-year-old head of Pembridge In...,None


In [16]:
import requests
import json

def get_response_content(prompt):
    # Define the URL and the request body
    url = "http://localhost:11434/api/chat"
    body = {
        "model": "llama3.2:3b",
        "messages": [
            {
                "role": "system",
                "content": "Act as a classification model. Answer the prompts with yes or no."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False
    }

    try:
        # Send the POST request
        response = requests.post(url, json=body)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Extract and return only the 'content' field from the response
            response = response.json().get('message', {}).get('content', "No content found")
            # print(response)
            extracted_response = response.split("</think>")[1].strip()
            return extracted_response
        else:
            return f"Failed to get a valid response. Status code: {response.status_code}, Response: {response.text}"

    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

In [17]:
prompt = tmp_train_df['prompt'][0]
prompt

'Sentence: Latest corporate unbundler reveals laid-back approach : Roland Franklin , who is leading a 697m pound break-up bid for DRG , talks to Frank Kane\nTarget Word: Latest (POS: ADJ, Word Index: 0, Word Sense: up to the immediate present; most recent or most up-to-date, definition: Near the end of a period of time.)\nIs the target word used metaphorically? Provide a yes or no answer without explanation.'

In [18]:
get_response_content(prompt)

IndexError: list index out of range

In [ ]:
from tqdm import tqdm
import time  # Optional, for implementing a delay between retries if needed

tqdm_progress_bar = tqdm(tmp_train_df.index[:])
for row_idx in tqdm_progress_bar:
    while True:  # Keep retrying until successful
        try:
            prompt = tmp_train_df.loc[row_idx, 'prompt']
            prediction = get_response_content(prompt)
            tmp_train_df.loc[row_idx, 'prediction'] = prediction
            tqdm_progress_bar.update(1)
            tqdm_progress_bar.set_description(prediction)
            break  # Break out of the retry loop once successful
        except Exception as e:
            print(f"Error processing row {row_idx}: {e}")
            # Optional: Add a small delay before retrying
    
    time.sleep(2)


# Convert Yes/No to labels

In [ ]:
tmp_train_df['predictionIdx'] = None

In [ ]:
for row_idx in tmp_train_df.index:
    prediction = tmp_train_df.loc[row_idx, 'prediction']
    
    if prediction is None:
        continue
    # convert to 1 or 0
    if "yes" in prediction.lower():
        predictionIdx = 1
    if "no" in prediction.lower():
        predictionIdx = 0
    tmp_train_df.loc[row_idx, 'predictionIdx'] = predictionIdx

In [ ]:
tmp_train_df[:262][['prompt', 'label', 'prediction']]

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# Calculate counts for each class
# label_counts = tmp_train_df[:262]['label'].value_counts()
# prediction_counts = tmp_train_df['predictionIdx'].value_counts()

# Confusion matrix components
TP = ((tmp_train_df['label'] == 1) & (tmp_train_df['predictionIdx'] == 1)).sum()
TN = ((tmp_train_df['label'] == 0) & (tmp_train_df['predictionIdx'] == 0)).sum()
FP = ((tmp_train_df['label'] == 0) & (tmp_train_df['predictionIdx'] == 1)).sum()
FN = ((tmp_train_df['label'] == 1) & (tmp_train_df['predictionIdx'] == 0)).sum()


conf_matrix = np.array([[TN, FP], [FN, TP]])



# Create confusion matrix
# Plotting the confusion matrix
fig, ax = plt.subplots(figsize=(6, 6))
cax = ax.matshow(conf_matrix, cmap="Blues")

# Adding color bar
plt.colorbar(cax)

# Setting axis labels and ticks
ax.set_xticklabels([''] + ['Predicted Negative', 'Predicted Positive'])
ax.set_yticklabels([''] + ['Actual Negative', 'Actual Positive'])

# Annotating the cells with counts
for (i, j), val in np.ndenumerate(conf_matrix):
    ax.text(j, i, f'{val}', ha='center', va='center', color='black')

# Titles and grid
plt.title('Confusion Matrix', pad=20)
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.show()